In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
feat = pd.read_csv('/content/drive/My Drive/ICCSP/feat.csv')
target = pd.read_csv('/content/drive/My Drive/ICCSP/target.csv')

X = feat.values

encoder = LabelEncoder()
y = encoder.fit_transform(target.values)

In [0]:
X = X.reshape((3333, 900))

blsmote = BorderlineSMOTE()
X, y = blsmote.fit_resample(X, y)

X = X.reshape((len(X), 18, 50))
y = to_categorical(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [0]:
model = Sequential()
model.add(Convolution1D(200, kernel_size=3, activation='relu', input_shape=(18, 50), name='m1_conv1'))
model.add(Dropout(0.5, name='m1_drop1'))
model.add(Convolution1D(128, kernel_size=3, activation='relu', name='m1_conv2'))
model.add(Dropout(0.5, name='m1_drop2'))
model.add(Convolution1D(64, kernel_size=3, activation='relu', name='m1_conv3'))
model.add(Dropout(0.5, name='m1_drop3'))
model.add(Flatten(name='m1_flatten1'))
model.add(Dropout(0.5, name='m1_drop4'))
model.add(Dense(64, activation='relu', name='m1_dense1'))
model.add(Dense(6, activation='softmax', name='m1_dense2'))

model.compile(Adam(lr=0.0003), loss='categorical_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=16)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3000, batch_size=64, callbacks=[es])